In [2]:
! pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 2.7 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 MB 270.1 kB/s eta 0:00:0000:0100:15


In [1]:
import pandas as pd
import mlflow
import mlflow.xgboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

In [3]:
mlflow.set_tracking_uri("http://mlflow-web:5000")
mlflow.set_experiment("experiment_heart_attack")

MlflowException: Cannot set a deleted experiment 'experiment_heart_attack' as the active experiment. You can restore the experiment, or permanently delete the experiment to create a new one.

In [4]:
mlflow.autolog()

2024/12/02 07:37:00 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/12/02 07:37:00 WARNING mlflow.utils.autologging_utils: MLflow xgboost autologging is known to be compatible with 1.4.2 <= xgboost <= 2.1.2, but the installed version is 2.1.3. If you encounter errors during autologging, try upgrading / downgrading xgboost to a compatible version, or try upgrading MLflow.
2024/12/02 07:37:00 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.


In [5]:
df = pd.read_csv('data/data-cleaned.csv')

# Define X and y if not already defined
X = df.drop("output", axis=1)
y = df["output"]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
model = XGBClassifier(max_depth=3, learning_rate=0.1, n_estimators=100)
model.fit(X_train, y_train)

predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

2024/12/02 07:37:45 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '8ccddfb6be9c4e5e831d46f651458335', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
2024/12/02 07:37:47 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/conda/lib/python3.11/site-packages/mlflow/types/utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https:/

🏃 View run fun-rook-739 at: http://mlflow-web:5000/#/experiments/1/runs/8ccddfb6be9c4e5e831d46f651458335
🧪 View experiment at: http://mlflow-web:5000/#/experiments/1


In [5]:
# # Set up MLflow tracking URI
# mlflow.set_tracking_uri("http://mlflow:5000")
# mlflow.set_experiment("experiment_heart_attack")

# mlflow.autolog()

# def training(max_depth, learning_rate, n_estimators):
#     with mlflow.start_run():
#         # Define model and hyperparameters
#         model = XGBClassifier(max_depth=max_depth, learning_rate=learning_rate, n_estimators=n_estimators)
#         model.fit(X_train, y_train)

#         # Evaluate the model
#         predictions = model.predict(X_test)
#         accuracy = accuracy_score(y_test, predictions)

#         # Log parameters and metrics
#         mlflow.log_param("max_depth", max_depth)
#         mlflow.log_param("learning_rate", learning_rate)
#         mlflow.log_param("n_estimators", n_estimators)
#         mlflow.log_metric("accuracy", accuracy)

#         # Log the model
#         mlflow.xgboost.log_model(model, artifact_path="models")

#         # Register the model
#         model_uri = f"runs:/{mlflow.active_run().info.run_id}/models"
#         model_name = "heart_attack_model"
#         registered_model = mlflow.register_model(model_uri, model_name)

#         print(f"Model registered with version: {registered_model.version}")

# # Run the training
# training(max_depth=3, 
#          learning_rate=0.1, 
#          n_estimators=100)

KeyboardInterrupt: 